In [1]:
pip install xgboost

   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/99.8 MB 2.4 MB/s eta 0:00:42
   ---------------------------------------- 0.7/99.8 MB 7.0 MB/s eta 0:00:15
    --------------------------------------- 1.4/99.8 MB 10.2 MB/s eta 0:00:10
    --------------------------------------- 2.2/99.8 MB 11.5 MB/s eta 0:00:09
   - -------------------------------------- 3.0/99.8 MB 12.6 MB/s eta 0:00:08
   - -------------------------------------- 3.6/99.8 MB 13.6 MB/s eta 0:00:08
   - -------------------------------------- 4.6/99.8 MB 14.7 MB/s eta 0:00:07
   -- ------------------------------------- 5.4/99.8 MB 14.5 MB/s eta 0:00:07
   -- ------------------------------------- 6.4/99.8 MB 15.7 MB/s eta 0:00:06
   -- ------------------------------------- 7.4/99.8 MB 16.4 MB/s eta 0:00:06
   --- ------------------------------------ 8.4/99.8 MB 16.8 MB/s eta 0:00:06
   -

In [19]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [20]:
# match your subgroups again
sub1 = pd.read_csv("C:/Users/jyson/dsc180ab/dsc180b-wi24-quarter2/processed_data/subgroup_1.csv").assign(subgroup = 1) # Complicated Diabetes
sub2 = pd.read_csv("C:/Users/jyson/dsc180ab/dsc180b-wi24-quarter2/processed_data/subgroup_2.csv").assign(subgroup = 2) # Cardiac
sub3 = pd.read_csv("C:/Users/jyson/dsc180ab/dsc180b-wi24-quarter2/processed_data/subgroup_3.csv").assign(subgroup = 3) # Hepatic and Addiction
sub4 = pd.read_csv("C:/Users/jyson/dsc180ab/dsc180b-wi24-quarter2/processed_data/subgroup_4.csv").assign(subgroup = 4) # Cardiopulmonary
sub5 = pd.read_csv("C:/Users/jyson/dsc180ab/dsc180b-wi24-quarter2/processed_data/subgroup_5.csv").assign(subgroup = 5) # Young
sub6 = pd.read_csv("C:/Users/jyson/dsc180ab/dsc180b-wi24-quarter2/processed_data/subgroup_6.csv").assign(subgroup = 6) # 
sub7 = pd.read_csv("C:/Users/jyson/dsc180ab/dsc180b-wi24-quarter2/processed_data/subgroup_7.csv").assign(subgroup = 7) # Cardiopulmonary

In [46]:
sub1[sub1['subject_id'] == 453]

,subject_id,hadm_id,rounded_age,admission_type_encoded,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,...,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression,age,subgroup
44,453,187961,80,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,80,1


In [61]:
sub_all = pd.concat([sub1, sub2, sub3, sub4, sub5, sub6, sub7], axis=0, ignore_index=False).drop(columns = ['rounded_age'])
len(sub_all['hadm_id'].unique())

48289

In [48]:
sub_all[sub_all['subject_id'] == 453]

,subject_id,admission_type_encoded,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,paralysis,other_neurological,...,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression,age,subgroup
44,453,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,80,1


In [38]:
# 'subgroup' is the target column
X = sub_all.drop('subgroup', axis = 1)
y = sub_all['subgroup']

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
# train the XGBoost model
y_train_adj = y_train - 1
y_test_adj = y_test - 1

dtrain = xgb.DMatrix(X_train, label=y_train_adj)
dtest = xgb.DMatrix(X_test, label=y_test_adj)


# Specify parameters for the XGBoost model
params = {
    'max_depth': 3,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 7  # the number of classes that exist in this datset
}

# Train the model
num_round = 100  # the number of training iterations
bst = xgb.train(params, dtrain, num_round)

In [41]:
# make predictions and evaluate the model
# Make prediction
preds = bst.predict(dtest)
best_preds = np.asarray([np.argmax(line) for line in preds])

# Evaluate the model
accuracy = accuracy_score(y_test, best_preds)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 0.66%


In [35]:
# Make probability predictions
probabilities = bst.predict(dtest)

In [69]:
# subject_id = 453's index
patient_probabilities = probabilities[44]
patient_probabilities.round()

array([0., 0., 1., 0., 0., 0., 0.], dtype=float32)